## R&D Capital Replication

#### Peyton Lewis

### Necessary Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

### Read in and Clean Fama French Data

In [2]:
# Read in the data and set the index to column renamed as DATE
fama_data = pd.read_csv('FFFactors2.csv')
fama_data = fama_data.set_index('Unnamed: 0')
fama_data.index.name = 'DATE'

# Transform fama date format to datetime object and reformat as year-month from format without a dash in between 
fama_data.index = pd.to_datetime(fama_data.index, format = '%Y%m')
fama_data.index = fama_data.index.strftime('%Y-%m')

# Divide all columns by 100 because they are in percent format originally 
fama_data = fama_data/100

### Read in and Clean Monthly Data

In [3]:
# Read in the monthly data and set date to datetime 
monthly_data = pd.read_csv('monthly_data.csv') 
monthly_data['date'] = pd.to_datetime(monthly_data['date'])
monthly_data.sort_values(by='date', inplace=True)

In [4]:
# Function to help remove non-numeric returns from monthly_data
def is_float(x):
    try:
        float(x)
    except ValueError:
        return False
    return True  # returns True if numeric 

In [5]:
# Apply numeric filter to monthly_data
numeric_filter = monthly_data['RET'].map(is_float)
monthly_data = monthly_data[numeric_filter]

# Drop any Nans for returns column and convert them to floats
monthly_data = monthly_data.dropna(subset = ['RET'])
monthly_data['RET'] = monthly_data['RET'].astype(float)

# Compute montly ME as shares outstanding * abs(price)
monthly_data['MONTHLY_ME'] = monthly_data['SHROUT'].astype(float) * abs(monthly_data['PRC'].astype(float))

# Shift MONTHLY_ME by one month and group by PERMNO so use ME at time t-1 in the value-weighting later
monthly_data = monthly_data.sort_values(['date','PERMNO'])
monthly_data['SHIFTED_MONTHLY_ME'] = monthly_data.groupby('PERMNO')['MONTHLY_ME'].shift(1)
monthly_data = monthly_data.dropna(subset = ['SHIFTED_MONTHLY_ME'])  # drop the Nas that result from shifting monthly ME (does not affect results substantially)

# Filter out returns < -1 and slice to be appropriate time window
returns_filter = (monthly_data['RET'] > -1) 
monthly_data = monthly_data[returns_filter]
monthly_data = monthly_data[(monthly_data['date'] >= '1975-01-01') & (monthly_data['date'] <= '2022-01-01')]

In [6]:
# Subtracting the RF rate from the montly returns at beginning so always using excess returns 

# Creating this date so that I can merge on the date in the Fama French data (same year-month format)
monthly_data['YEAR_MONTH'] = monthly_data['date'].apply(lambda x: x.strftime('%Y-%m'))

# Merge fama and monthly data
monthly_data = monthly_data.merge(fama_data, left_on = 'YEAR_MONTH', right_on = 'DATE', how = 'left')

# Subtract the RF rate from all returns
monthly_data['ORIG_RET'] = monthly_data['RET'] * 1   # kept to check to see if rf was subtracted correctly
monthly_data['RET'] = monthly_data['ORIG_RET'] - monthly_data['RF']  # this column will be used in remainder of calculations 

### Reading in and Cleaning the Annual Data

In [7]:
# Read in the annual financial data and set date to datetime 
annual_financials = pd.read_csv('annual_financials.csv')
annual_financials['datadate'] = pd.to_datetime(annual_financials['datadate'])

# Sort by date
annual_financials.sort_values(by='datadate', inplace=True)

In [8]:
# Filtering the Data

# Filter out the sic codes between 6000 and 6999
sic_filter = (annual_financials['sic'] >= 6000) & (annual_financials['sic'] <= 6999)
annual_financials = annual_financials[~sic_filter]

# Filter out fund exchange codes to be between 11 and 19 only
exch_filter = (annual_financials['exchg'] >= 11) & (annual_financials['exchg'] <= 19)
annual_financials = annual_financials[exch_filter]

# Fill NaN with 0 in xrd column 
annual_financials['xrd'].fillna(0, inplace=True)

# Create ME Column (shares * price)
annual_financials['ME'] = annual_financials['csho'] * annual_financials['prcc_f']

# Drop the columns that are not needed
annual_financials = annual_financials.drop(['indfmt', 'consol', 'popsrc', 'datafmt', 'costat'], axis = 1)

### Look Ahead Bias: Shift all dates by 90 days in annual financials

In [9]:
# shift all datadates by 90 days in annual financials
annual_financials['datadate'] = annual_financials['datadate'] + pd.DateOffset(days=90)

# Filter annual_financials for time 1975-2022 
annual_financials = annual_financials[(annual_financials['datadate'] >= '1975-01-01') & (annual_financials['datadate'] <= '2022-01-01')]

# Drop Nans in ME
annual_financials = annual_financials.dropna(subset = ['ME'])

### Functions to Fix Fyear (annual_financials) and (monthly) to correctly merge

In [10]:
# Function for the annual data the creates YEAR column  
def get_year(date):
    if date.month >= 4:
        return date.year + 1
    else:
        return date.year 

# Function for the monthly data the creates YEAR column 
def get_year_month(date):
    if date.month >= 4:
        return date.year
    else:
        return date.year - 1

# Apply both functions to the data
monthly_data['YEAR'] = monthly_data['date'].apply(get_year_month)
annual_financials['YEAR'] = annual_financials['datadate'].apply(get_year)

### Methodology Description: 
The thought process behind this was that the fyear given in the annual financials would not work for my goal of being able to form the portfolio once a year after the data was shifted 90 days forward. I decided to build my portfolio once a year on every April 1. To get the correct portfolio reconstruction year for every stock, which I defined to be the April 1, YEAR in which the portfolio was made, I created this 'YEAR' column in the functions shown above. This procedure looked different for the annual and monthly data.  <br><br>
For the annual data, the following process was used. For dates (post 90 day shift) with months of January, February, or March (months < 4), their 'YEAR' was just the same as the year in their datadate column, because those financials would be available prior to April 1, when that YEAR's portfolio was being built. For months April - December (month >= 4), one was added to their datadate year, because those financials would be the most recently available for the next year's portfolio construction. This was the method to make this 'YEAR' column in the annual financials data. <br><br>
In the monthly data, a similar process was used to create the same 'YEAR' column, but with a slightly different logic. If the month was >= 4, YEAR was set equal to the same year of its date and if month < 4, YEAR was set to its date's year - 1. The reason for this was as follows: once a portfolio is constructed for a YEAR, like 2021, it needs to track monthly returns for the next 12 months. For the months April - December (month >= 4), we want these returns available for the 2021 portfolio, which is the year they have in their date. For the months of Jan - March (month < 4) of the next year right before the next portfolio is made, we need those monthly returns, which are in the year 2022. To make them line up with the 2021 YEAR's portfolio, one needed to be subtracted from their dates' year (i.e. 2022 - 1 = 2021). Therefore, for these months, I subtracted 1 from their dates' year to create the 'YEAR' variable. This fascilitated the merging later on. 
<br><br>
Additionally, I chose to rebuild the portfolio every April 1, because the majority of firms had financial year ends in December. This is because with the 90 day shift, their annuals were ready the following 3/31 and would be the "most recent" financials available for each portfolio construction on 4/1/YEAR. This assumption was made so that for a majority of the firms, I would have the most recently available financials, and for fewer firms I would be using more "stale" financial data. 

### Disclaimer:
Later on in the portfolio construction process, when there is a time window that does not begin exactly at 4/1/YEAR, the assumption that I am making is that that portfolio was indeed constructed on the relevant 4/1/YEAR, and we are tracking its returns for the subsequent months that fall into the desired time window. For example, in the time range 1981.07 - 1999.12, the assumption is that the 1981 portfolio was constructed on 4/1/1981, but we are tracking its returns from July onward until the end of the March of 1984, right before the 1984 porfolio is made. Likewise, in the time slice 2000.01 - 2012.12, for example, the assumption is that the 1999 portfolio (constructed on 4/1/1999), has returns that are being tracked in this time period from January 2000 - March 2000, right before the 2001 portfolio is constructed on 4/1/2001. This assumption is used throughout the portfolio construction processes so that the time windows of the assignment would be aligned with the results, even though I build the portfolios (sort by RD) on every April 1, YEAR.

### Compute RDC and RDC/ME for firms

In [11]:
# Set negative xrd values to 0 
# I made this assumption because I found 7 rows in the annual data with a negative xrd value; I set these to 0 and proceeded with calculations
# I felt as though this would not skew the results drastically; I actually tried both ways in the code and the answers were only off in the 0.0001 place
# I decided to keep the code with the negative xrd values set to 0 because a negative R&D expenditure does not make sense
# I checked the documentation in WRDS and it does not say anything about negative values for xrd, so I continued with my assumption 
annual_financials['xrd'] = annual_financials['xrd'].apply(lambda x: 0 if x < 0 else x)

In [12]:
# Created rd1 - rd4 using groupby and .shift()
rd1 = annual_financials.groupby('GVKEY')['xrd'].shift(1)
rd2 = annual_financials.groupby('GVKEY')['xrd'].shift(2)
rd3 = annual_financials.groupby('GVKEY')['xrd'].shift(3)
rd4 = annual_financials.groupby('GVKEY')['xrd'].shift(4)

# Added rd1-rd4 as columns to annual_financials for ease of computing 
annual_financials['rd1'] = rd1
annual_financials['rd2'] = rd2
annual_financials['rd3'] = rd3
annual_financials['rd4'] = rd4

# Set rd1, rd2, rd3 and rd4 Nans to 0
annual_financials['rd1'].fillna(0, inplace=True)
annual_financials['rd2'].fillna(0, inplace=True)
annual_financials['rd3'].fillna(0, inplace=True)
annual_financials['rd4'].fillna(0, inplace=True)

# Compute rdc 
annual_financials['rdc'] = annual_financials['xrd'] + 0.8 * annual_financials['rd1'] + 0.6 * annual_financials['rd2'] + 0.4 * annual_financials['rd3'] + 0.2 * annual_financials['rd4']

# Compute rdc/ME (normalizing by the market capitalization)
annual_financials['rdc/ME'] = annual_financials['rdc'] / annual_financials['ME']

### Merging Annual and Monthly Data 

In [13]:
# Merge annual_financials with monthly_data on lpermno and year and permno and year
merge = monthly_data.merge(annual_financials, how = 'inner', left_on = ['PERMNO', 'YEAR'], right_on = ['LPERMNO', 'YEAR'])

# Remove unneccesary columns 
merge.drop(columns = ['EXCHCD', 'SICCD', 'TICKER', 'LPERMNO', 'conm', 'curcd', 'csho', 'exchg', 'mkvalt', 'sic', 'fyr'], inplace = True)
merge.drop(columns = ['rd1', 'rd2', 'rd3', 'rd4', 'rdc', 'Mkt-RF', 'SMB', 'HML', 'RF'], inplace = True)

# Rename columns to more clear names for later use
merge.rename({'datadate': 'ANNUAL_DATE'}, axis = 1, inplace = True)
merge.rename({'date': 'MONTHLY_DATE'}, axis = 1, inplace = True)
merge.rename({'prcc_f': 'ANNUAL_PRICE'}, axis = 1, inplace = True)
merge.rename({'PRC': 'MONTHLY_PRICE'}, axis = 1, inplace = True)

# Drop duplicates for permno and monthly date (sometimes it was keeping 2 records for a firm in  YEAR, but we want to keep the one with latest financial data each time)
# For example for the YEAR 1983, there was a record for a firm for 3/31/1983 and 5/31/1982, which is the same reconsitution YEAR (1983), but we want to keep
# the record with the most recently available financial data as of reconstruction on 4/1/YEAR, which is the 3/31/1983 row 
# This may be the result of a firm switching its fiscal year end midway
merge.sort_values(['PERMNO', 'MONTHLY_DATE', 'ANNUAL_DATE'], inplace = True)
merge.drop_duplicates(subset = ['PERMNO', 'MONTHLY_DATE'], keep = 'last', inplace = True)

### Portfolio Construction and Returns Function

In [14]:
# Function that will go once each 'YEAR' and make RD and nonRD dataframe, 
# then add indicator for porfolio (1-5 and NonRD) and then make df of monthly portfolio returns 
# for equally weighted or value weighted returns 

def make_portfolios(merge, start_date, end_date, type_weight) :

    # Filter for the year
    df = merge[(merge['MONTHLY_DATE'] >= start_date) & (merge['MONTHLY_DATE'] <= end_date)]

    # Set up empty df for returns
    portfolio_returns = pd.DataFrame()
    monthly_me = pd.DataFrame()

    # Years to loop through 
    total_years = df['YEAR'].unique()
    
    # Sort years 
    total_years = np.sort(total_years)

    # Loop through the years
    for year in total_years: 
       
        # Filter for the year
        df_year = df[df['YEAR'] == year]

        # RD firms will not have zero measure here; NonRD firms will have zero measure
        rd = df_year[df_year['rdc/ME'] > 0]
        non_rd = df_year[df_year['rdc/ME'] == 0]

        # Sort by rdc/ME to make portfolios
        rd = rd.sort_values(by = ['rdc/ME'], ascending = False)

        # Cut rd firms into 5 bins
        rd['bin'] = pd.qcut(rd['rdc/ME'], 5, labels = [1,2,3,4,5])
        non_rd['bin'] = 'NonRD'

        # Put dataframes back together
        joined_df = pd.concat([rd, non_rd])

       
        # Equal weighted portfolios
        if type_weight == 'equal' : 

            # Calculate portfolio returns and add to df
            # Grouped by monthly_date and bin to get an average return for each portfolio for each month in df 
            # Need to unstack because had hierarchical indices from groupby
            portfolio_returns = portfolio_returns.append(joined_df.groupby(['MONTHLY_DATE', 'bin'])['RET'].mean().unstack())    

        # Value weighted portfolios 
        elif type_weight == 'value' : 

            # Make separate dfs by bin
            bin_low = joined_df[joined_df['bin'] == 1]
            bin_2 = joined_df[joined_df['bin'] == 2]
            bin_3 = joined_df[joined_df['bin'] == 3]
            bin_4 = joined_df[joined_df['bin'] == 4]
            bin_high = joined_df[joined_df['bin'] == 5]
            bin_nonRD = joined_df[joined_df['bin'] == 'NonRD']

            # Dataframe of the sum of the lagged ME for the stocks in each portfolio to use as denominator of weights in each period
            # Group by monthly_date and bin, then take sum of shifted monthly ME for that combination and store in the df
            # Unstack the hierarchical indices 
            monthly_me = monthly_me.append(joined_df.groupby(['MONTHLY_DATE', 'bin'])['SHIFTED_MONTHLY_ME'].sum().unstack()) 


            # Bin 1

            # Get where the index is equal to the date and return the value of the summed ME for that monthly date for that bin ; make a new column of this series
            # The summed ME will be used as the denominator in the weights calculations later 
            bin_low['MONTHLY_ME_TOTAL'] = bin_low.apply(lambda x: monthly_me.loc[x['MONTHLY_DATE'], 1], axis = 1)
            
            # Compute the weight for each stock as its lagged ME / sum of lagged ME's in that portfolio on that monthly_date
            bin_low['weight'] = bin_low['SHIFTED_MONTHLY_ME'] / bin_low['MONTHLY_ME_TOTAL']


            # Bin 2

            # Get where the index is equal to the date and return the value of the summed ME for that monthly date for that bin ; make a new column of this series
            # The summed ME will be used as the denominator in the weights calculations later 
            bin_2['MONTHLY_ME_TOTAL'] = bin_2.apply(lambda x: monthly_me.loc[x['MONTHLY_DATE'], 2], axis = 1)
            
            # Compute the weight for each stock as its lagged ME / sum of lagged ME's in that portfolio on that monthly date
            bin_2['weight'] = bin_2['SHIFTED_MONTHLY_ME'] / bin_2['MONTHLY_ME_TOTAL']


            # Bin 3
            
            # Get where the index is equal to the date and return the value of the summed ME for that monthly date for that bin ; make a new column of this series
            # The summed ME will be used as the denominator in the weights calculations later 
            bin_3['MONTHLY_ME_TOTAL'] = bin_3.apply(lambda x: monthly_me.loc[x['MONTHLY_DATE'], 3], axis = 1)
            
            # Compute the weight for each stock as its lagged ME / sum of lagged ME's in that portfolio on that monthly date
            bin_3['weight'] = bin_3['SHIFTED_MONTHLY_ME'] / bin_3['MONTHLY_ME_TOTAL']


            # Bin 4
            
            # Get where the index is equal to the date and return the value of the summed ME for that monthly date for that bin ; make a new column of this series
            # The summed ME will be used as the denominator in the weights calculations later 
            bin_4['MONTHLY_ME_TOTAL'] = bin_4.apply(lambda x: monthly_me.loc[x['MONTHLY_DATE'], 4], axis = 1)
            
            # Compute the weight for each stock as its lagged ME / sum of lagged ME's in that portfolio on that monthly date
            bin_4['weight'] = bin_4['SHIFTED_MONTHLY_ME'] / bin_4['MONTHLY_ME_TOTAL']


            # Bin 5
            
            # Get where the index is equal to the date and return the value of the summed ME for that monthly date for that bin ; make a new column of this series
            # The summed ME will be used as the denominator in the weights calculations later 
            bin_high['MONTHLY_ME_TOTAL'] = bin_high.apply(lambda x: monthly_me.loc[x['MONTHLY_DATE'], 5], axis = 1)
            
            # Compute the weight for each stock as its lagged ME / sum of lagged ME's in that portfolio on that monthly date
            bin_high['weight'] = bin_high['SHIFTED_MONTHLY_ME'] / bin_high['MONTHLY_ME_TOTAL']


            # NonRD Bin
            
            # Get where the index is equal to the date and return the value of the summed ME for that monthly date for that bin ; make a new column of this series
            # The summed ME will be used as the denominator in the weights calculations later 
            bin_nonRD['MONTHLY_ME_TOTAL'] = bin_nonRD.apply(lambda x: monthly_me.loc[x['MONTHLY_DATE'], 'NonRD'], axis = 1)
            
            # Compute the weight for each stock as its lagged ME / sum of lagged ME's in that portfolio on that monthly date
            bin_nonRD['weight'] = bin_nonRD['SHIFTED_MONTHLY_ME'] / bin_nonRD['MONTHLY_ME_TOTAL']

            
            # Put dataframes back together
            joined_df = pd.concat([bin_low, bin_2, bin_3, bin_4, bin_high, bin_nonRD])

            # Make new column with weight * return
            joined_df['WEIGHTED_RET'] = joined_df['RET'] * joined_df['weight']

            # Calculate portfolio returns by grouping by monthly date and bin to get a total weighted return for that portfolio for that time
            # Creates df that is returned that has for every month, each portfolio's total weighted return
            portfolio_returns = portfolio_returns.append(joined_df.groupby(['MONTHLY_DATE', 'bin'])['WEIGHTED_RET'].sum().unstack())
        
        else: 
            print('Type must be equal or value')

 
    return portfolio_returns

<br>

### **1. Calculate Equal-Weighted Returns for R&D and Non-R&D Firms.**

#### Equally-Weighted Results

In [15]:
# Perform for various time periods
results_equal_1981_1999 = make_portfolios(merge, '1981-07-01', '1999-12-31', 'equal')
results_equal_2000_2012 = make_portfolios(merge, '2000-01-01', '2012-12-31', 'equal') 
results_equal_2013_2021 = make_portfolios(merge, '2013-01-01', '2021-12-31', 'equal') 
results_equal_1981_2012 = make_portfolios(merge, '1981-07-01', '2012-12-31', 'equal')
results_equal_1981_2021 = make_portfolios(merge, '1981-07-01', '2021-12-31', 'equal')

In [16]:
# 1981 - 1999

# Collapses the portfolio_returns df from the function to a series of mean returns for each portfolio for the time range
results_equal_1981_1999_summary = pd.DataFrame(results_equal_1981_1999.mean(axis = 0)).T
results_equal_1981_1999_summary = results_equal_1981_1999_summary * 100  # Converts to percentage return 
results_equal_1981_1999_summary = results_equal_1981_1999_summary.rename(columns = {1: 'Low', 5: 'High'})
results_equal_1981_1999_summary['Time'] = '1981.07 - 1999.12'
# set time to index
results_equal_1981_1999_summary = results_equal_1981_1999_summary.set_index('Time')


# 2000 - 2012

# Collapses the portfolio_returns df from the function to a series of mean returns for each portfolio for the time range
results_equal_2000_2012_summary = pd.DataFrame(results_equal_2000_2012.mean(axis = 0)).T
results_equal_2000_2012_summary = results_equal_2000_2012_summary * 100
results_equal_2000_2012_summary = results_equal_2000_2012_summary.rename(columns = {1: 'Low', 5: 'High'})
results_equal_2000_2012_summary['Time'] = "2000.01 - 2012.12"
# set time to index
results_equal_2000_2012_summary = results_equal_2000_2012_summary.set_index('Time')


# 1981 - 2012

# Collapses the portfolio_returns df from the function to a series of mean returns for each portfolio for the time range
results_equal_1981_2012_summary = pd.DataFrame(results_equal_1981_2012.mean(axis = 0)).T
results_equal_1981_2012_summary = results_equal_1981_2012_summary * 100
results_equal_1981_2012_summary = results_equal_1981_2012_summary.rename(columns = {1: 'Low', 5: 'High'})
results_equal_1981_2012_summary['Time'] = "1981.07 - 2012.12"
# set time to index
results_equal_1981_2012_summary = results_equal_1981_2012_summary.set_index('Time')

In [17]:
# Concatenate all dataframes
results_equal_tog = pd.concat([results_equal_1981_2012_summary, results_equal_1981_1999_summary, results_equal_2000_2012_summary])
# results_equal_tog.to_csv('results_equal_tog.csv')
results_equal_tog

bin,Low,2,3,4,High,NonRD
Time,,,,,,
1981.07 - 2012.12,0.244474,0.592820,0.869259,1.120119,1.769685,0.706033
1981.07 - 1999.12,0.242229,0.598276,0.865817,1.122465,1.722986,0.569318
2000.01 - 2012.12,0.247645,0.581299,0.873003,1.113901,1.836796,0.900588


<br>

### **2. Long-Short Portfolio - CAPM Alpha, FF3 Alpha, and SR (Equal-Weighted)**

#### (i) CAPM Alpha

In [18]:
# Rename cols, get zero cost portfolio (High - Low)
results_equal_1981_2012 = results_equal_1981_2012.rename(columns = {1: 'Low', 5: 'High'})
results_equal_1981_2012['H-L'] = results_equal_1981_2012['High'] - results_equal_1981_2012['Low']
# Create Year-Month column to merge fama data with so it has the same format
results_equal_1981_2012['YEAR_MONTH'] = results_equal_1981_2012.index.year.astype(str) + '-' + results_equal_1981_2012.index.month.astype(str).str.zfill(2)

# Merge fama and portfolio returns on Year-Month
merged_equal_fama = results_equal_1981_2012.merge(fama_data, left_on = 'YEAR_MONTH', right_on = 'DATE')
merged_equal_fama = merged_equal_fama.set_index('YEAR_MONTH')

In [19]:
# Regressing the H-L on the Mkt-RF
X = merged_equal_fama['Mkt-RF']
y = merged_equal_fama['H-L']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
print(model.summary())

# T stat and coeff on the constant
t_stat = model.tvalues[0]
coef = model.params[0]

                            OLS Regression Results                            
Dep. Variable:                    H-L   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     3.012
Date:                Tue, 22 Nov 2022   Prob (F-statistic):             0.0834
Time:                        19:16:25   Log-Likelihood:                 609.49
No. Observations:                 378   AIC:                            -1215.
Df Residuals:                     376   BIC:                            -1207.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0147      0.003      5.867      0.0

In [20]:
print('Equally Weighted Portfolio Results (1981-2012)')
print()
print('Monthly CAPM Alpha: ' + str(round(coef * 100, 3)) + '%')
print('t-statistic:', round(t_stat, 3))

Equally Weighted Portfolio Results (1981-2012)

Monthly CAPM Alpha: 1.471%
t-statistic: 5.867


#### (ii) FF3 Alpha

In [21]:
# FF3 Factor alpha
# Regressing H-L on Mkt-RF, SMB, and HML
X2 = merged_equal_fama[['Mkt-RF', 'SMB', 'HML']]
y2 = merged_equal_fama['H-L']
X2 = sm.add_constant(X2)
model2 = sm.OLS(y2, X2).fit()
print(model2.summary())

# T stat and coeff on the constant
t_stat2 = model2.tvalues[0]
coef2 = model2.params[0]

                            OLS Regression Results                            
Dep. Variable:                    H-L   R-squared:                       0.233
Model:                            OLS   Adj. R-squared:                  0.227
Method:                 Least Squares   F-statistic:                     37.96
Date:                Tue, 22 Nov 2022   Prob (F-statistic):           1.96e-21
Time:                        19:16:25   Log-Likelihood:                 658.21
No. Observations:                 378   AIC:                            -1308.
Df Residuals:                     374   BIC:                            -1293.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0144      0.002      6.417      0.0

In [22]:
print('Equally Weighted Portfolio Results (1981-2012)')
print()
print('Monthly FF3 Alpha: ' + str(round(coef2 * 100, 3)) + '%')
print('t-statistic:', round(t_stat2, 3))

Equally Weighted Portfolio Results (1981-2012)

Monthly FF3 Alpha: 1.438%
t-statistic: 6.417


#### (iii) Annualized Sharpe Ratio

In [23]:
# Creating the series of monthly returns of zero cost portfolio 
long_short = merged_equal_fama['H-L']

# Getting the mean and standard deviation of these returns 
mean_return = long_short.mean()
std_dev = long_short.std()

# Monthly and annual sharpe ratio
monthly_sharpe_ratio = mean_return / std_dev
annualized_sharpe_ratio = monthly_sharpe_ratio * np.sqrt(12)
print('Annualized Sharpe Ratio:', round(annualized_sharpe_ratio, 3))

Annualized Sharpe Ratio: 1.089


<br>

### **3. Repeat Steps 1, 2 for Value-Weighted Portfolios (Returns and Alphas)**

### Value-Weighted Results

In [24]:
# Perform for time periods
results_value_1981_1999 = make_portfolios(merge, '1981-07-01', '1999-12-31', 'value')
results_value_2000_2012 = make_portfolios(merge, '2000-01-01', '2012-12-31', 'value') 
results_value_2013_2021 = make_portfolios(merge, '2013-01-01', '2021-12-31', 'value') 
results_value_1981_2012 = make_portfolios(merge, '1981-07-01', '2012-12-31', 'value')
results_value_1981_2021 = make_portfolios(merge, '1981-07-01', '2021-12-31', 'value')

In [25]:
# 1981 - 1999

# Collapses the portfolio_returns df from the function to a series of mean returns for each portfolio for the time range
results_value_1981_1999_summary = pd.DataFrame(results_value_1981_1999.mean(axis = 0)).T
results_value_1981_1999_summary = results_value_1981_1999_summary * 100     # Converting it to percentage return 
results_value_1981_1999_summary = results_value_1981_1999_summary.rename(columns = {1: 'Low', 5: 'High'})
results_value_1981_1999_summary['Time'] = '1981.07 - 1999.12'
# set time to index
results_value_1981_1999_summary = results_value_1981_1999_summary.set_index('Time')


# 2000 - 2012

# Collapses the portfolio_returns df from the function to a series of mean returns for each portfolio for the time range
results_value_2000_2012_summary = pd.DataFrame(results_value_2000_2012.mean(axis = 0)).T
results_value_2000_2012_summary = results_value_2000_2012_summary * 100
results_value_2000_2012_summary = results_value_2000_2012_summary.rename(columns = {1: 'Low', 5: 'High'})
results_value_2000_2012_summary['Time'] = "2000.01 - 2012.12"
# set time to index
results_value_2000_2012_summary = results_value_2000_2012_summary.set_index('Time')


# 1981 - 2012

# Collapses the portfolio_returns df from the function to a series of mean returns for each portfolio for the time range
results_value_1981_2012_summary = pd.DataFrame(results_value_1981_2012.mean(axis = 0)).T
results_value_1981_2012_summary = results_value_1981_2012_summary * 100
results_value_1981_2012_summary = results_value_1981_2012_summary.rename(columns = {1: 'Low', 5: 'High'})
results_value_1981_2012_summary['Time'] = "1981.07 - 2012.12"
# set time to index
results_value_1981_2012_summary = results_value_1981_2012_summary.set_index('Time')

In [26]:
# Concatenate all dataframes
results_value_tog = pd.concat([results_value_1981_2012_summary, results_value_1981_1999_summary, results_value_2000_2012_summary])
# results_value_tog.to_csv('value_12.csv')
results_value_tog

bin,Low,2,3,4,High,NonRD
Time,,,,,,
1981.07 - 2012.12,0.438976,0.607080,0.664502,0.886940,0.966982,0.551287
1981.07 - 1999.12,0.675160,0.994105,1.055652,1.082074,1.193141,0.767481
2000.01 - 2012.12,0.100742,0.055259,0.109926,0.619786,0.646435,0.243628


### Long-Short Portfolio - CAPM Alpha, FF3 Alpha, and SR (Equal-Weighted)

#### (i) CAPM Alpha

In [27]:
# Rename cols, get zero cost portfolio (High - Low)
results_value_1981_2012 = results_value_1981_2012.rename(columns = {1: 'Low', 5: 'High'})
results_value_1981_2012['H-L'] = results_value_1981_2012['High'] - results_value_1981_2012['Low']
# Create Year-Month column to merge fama data with 
results_value_1981_2012['YEAR_MONTH'] = results_value_1981_2012.index.year.astype(str) + '-' + results_value_1981_2012.index.month.astype(str).str.zfill(2)

# Merge fama and portfolio returns
merged_value_fama = results_value_1981_2012.merge(fama_data, left_on = 'YEAR_MONTH', right_on = 'DATE')
merged_value_fama = merged_value_fama.set_index('YEAR_MONTH')

In [28]:
# Regressing the H-L on the Mkt-RF
X_v = merged_value_fama['Mkt-RF']
y_v = merged_value_fama['H-L']
X_v = sm.add_constant(X_v)
model_v = sm.OLS(y_v, X_v).fit()
print(model_v.summary())

# T stat and coeff on the constant
t_stat_v = model_v.tvalues[0]
coef_v = model_v.params[0]

                            OLS Regression Results                            
Dep. Variable:                    H-L   R-squared:                       0.102
Model:                            OLS   Adj. R-squared:                  0.100
Method:                 Least Squares   F-statistic:                     42.78
Date:                Tue, 22 Nov 2022   Prob (F-statistic):           2.01e-10
Time:                        19:18:38   Log-Likelihood:                 614.77
No. Observations:                 378   AIC:                            -1226.
Df Residuals:                     376   BIC:                            -1218.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0033      0.002      1.325      0.1

In [29]:
print('Value Weighted Portfolio Results (1981-2012)')
print()
print('Monthly CAPM Alpha: ' + str(round(coef_v * 100, 3)) + '%')
print('t-statistic:', round(t_stat_v, 3))

Value Weighted Portfolio Results (1981-2012)

Monthly CAPM Alpha: 0.328%
t-statistic: 1.325


#### (ii) FF3 Alpha

In [30]:
# FF3 Factor alpha
# Regressing H-L on Mkt-RF, SMB, and HML
X_v_2 = merged_value_fama[['Mkt-RF', 'SMB', 'HML']]
y_v_2 = merged_value_fama['H-L']
X_v_2 = sm.add_constant(X_v_2)
model_v_2 = sm.OLS(y_v_2, X_v_2).fit()
print(model_v_2.summary())

# T stat and coeff on the constant
t_stat_v_2 = model_v_2.tvalues[0]
coef_v_2 = model_v_2.params[0]

                            OLS Regression Results                            
Dep. Variable:                    H-L   R-squared:                       0.322
Model:                            OLS   Adj. R-squared:                  0.316
Method:                 Least Squares   F-statistic:                     59.13
Date:                Tue, 22 Nov 2022   Prob (F-statistic):           2.64e-31
Time:                        19:18:39   Log-Likelihood:                 667.77
No. Observations:                 378   AIC:                            -1328.
Df Residuals:                     374   BIC:                            -1312.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0010      0.002      0.469      0.6

In [31]:
print('Value Weighted Portfolio Results (1981-2012)')
print()
print('Monthly FF3 Alpha: ' + str(round(coef_v_2 * 100, 3)) + '%')
print('t-statistic:', round(t_stat_v_2, 3))

Value Weighted Portfolio Results (1981-2012)

Monthly FF3 Alpha: 0.102%
t-statistic: 0.469


#### (iii) Annualized Sharpe Ratio

In [32]:
# Series of monthly zero-cost portfolio returns 
long_short = merged_value_fama['H-L']

# Getting mean and std dev of returns
mean_return = long_short.mean()
std_dev = long_short.std()

# Monthly and annual SR
monthly_sharpe_ratio = mean_return / std_dev
annualized_sharpe_ratio = monthly_sharpe_ratio * np.sqrt(12)
print('Annualized Sharpe Ratio: ', round(annualized_sharpe_ratio, 3))

Annualized Sharpe Ratio:  0.364


<br>

### **4. Repeat Steps 1, 2 for Value-Weighted Portfolios (Returns and Alphas) - Excluding 1000 Largest Firms**

In [33]:
# Same function as was defined above, but now just dropping top 1000 value weighted firms each year before RD and NonRD split

def make_portfolios_1000(merge, start_date, end_date, type_weight) :

    # Filter for the year
    df = merge[(merge['MONTHLY_DATE'] >= start_date) & (merge['MONTHLY_DATE'] <= end_date)]

    # Set up empty df for returns
    portfolio_returns = pd.DataFrame()
    monthly_me = pd.DataFrame()

    # Years to loop through in portfolio creation 
    total_years = df['YEAR'].unique()
    
    # Sort years 
    total_years = np.sort(total_years)

    # Loop through the years
    for year in total_years: 
       
        # Filter for the year
        df_year = df[df['YEAR'] == year]

        # In df_year, sort by 'ME and remove the top 1000 permnos
        df_year = df_year.sort_values(by = 'ME', ascending = False)
        # Group by permno and take mean of ME (should have 12 records per permno for each YEAR with same ME value, so mean will return that value)
        group_df = df_year.groupby('PERMNO')['ME'].mean()
        group_df.sort_values(ascending = False, inplace = True)
        
        # Get a list of top 1000 permnos in index
        top_1000 = group_df.index[:1000]
        
        # Exclude these permnos
        df_year2 = df_year[~df_year['PERMNO'].isin(top_1000)]

        # Split into RD and NonRD dfs
        rd = df_year2[df_year2['rdc/ME'] > 0]
        non_rd = df_year2[df_year2['rdc/ME'] == 0]

        # Sort by rdc/ME
        rd = rd.sort_values(by = ['rdc/ME'], ascending = False)

        # Cut rd firms into 5 bins
        rd['bin'] = pd.qcut(rd['rdc/ME'], 5, labels = [1,2,3,4,5])
        non_rd['bin'] = 'NonRD'

        # Put dataframes back together
        joined_df = pd.concat([rd, non_rd])


        # Equal weighted portfolio returns 
        if type_weight == 'equal' : 
            
            # Calculate portfolio returns and add to df
            # Grouped by monthly_date and bin to get an average return for each bin for each month in df
            # This part will actually NOT get used because the top 1000 dropped is only for value weighted portfolios
            portfolio_returns = portfolio_returns.append(joined_df.groupby(['MONTHLY_DATE', 'bin'])['RET'].mean().unstack())    

        # Value weighted portfolio returns 
        elif type_weight == 'value' : 
            
            # Make separate dfs by bin
            bin_low = joined_df[joined_df['bin'] == 1]
            bin_2 = joined_df[joined_df['bin'] == 2]
            bin_3 = joined_df[joined_df['bin'] == 3]
            bin_4 = joined_df[joined_df['bin'] == 4]
            bin_high = joined_df[joined_df['bin'] == 5]
            bin_nonRD = joined_df[joined_df['bin'] == 'NonRD']

            # Dataframe of the sum of the lagged ME for proper stocks in each portfolio to use as denominator of weights in each portfolio
            # Group by monthly_date and bin, then take sum of shifted monthly ME for that combination and store in the df
            monthly_me = monthly_me.append(joined_df.groupby(['MONTHLY_DATE', 'bin'])['SHIFTED_MONTHLY_ME'].sum().unstack()) 
    
            
            # Bin 1

            # Get where the index is equal to the monthly date and return the value of the summed ME for that monthly date for that bin ; make a new column of this series
            # The summed ME will be used as the denominator in the weights calculations later 
            bin_low['MONTHLY_ME_TOTAL'] = bin_low.apply(lambda x: monthly_me.loc[x['MONTHLY_DATE'], 1], axis = 1)
            
            # Compute the weight for each stock as its lagged ME / sum of lagged MEs in that portfolio on that month_date
            bin_low['weight'] = bin_low['SHIFTED_MONTHLY_ME'] / bin_low['MONTHLY_ME_TOTAL']


            # Bin 2
            
            # Get where the index is equal to the monthly date and return the value of the summed ME for that monthly date for that bin ; make a new column of this series
            # The summed ME will be used as the denominator in the weights calculations later 
            bin_2['MONTHLY_ME_TOTAL'] = bin_2.apply(lambda x: monthly_me.loc[x['MONTHLY_DATE'], 2], axis = 1)
            
            # Compute the weight for each stock as its lagged ME / sum of lagged MEs in that portfolio on that month_date
            bin_2['weight'] = bin_2['SHIFTED_MONTHLY_ME'] / bin_2['MONTHLY_ME_TOTAL']


            # Bin 3
            
            # Get where the index is equal to the monthly date and return the value of the summed ME for that monthly date for that bin ; make a new column of this series
            # The summed ME will be used as the denominator in the weights calculations later 
            bin_3['MONTHLY_ME_TOTAL'] = bin_3.apply(lambda x: monthly_me.loc[x['MONTHLY_DATE'], 3], axis = 1)
            
            # Compute the weight for each stock as its lagged ME / sum of lagged MEs in that portfolio on that month_date
            bin_3['weight'] = bin_3['SHIFTED_MONTHLY_ME'] / bin_3['MONTHLY_ME_TOTAL']

            
            # Bin 4
            
            # Get where the index is equal to the monthly date and return the value of the summed ME for that monthly date for that bin ; make a new column of this series
            # The summed ME will be used as the denominator in the weights calculations later 
            bin_4['MONTHLY_ME_TOTAL'] = bin_4.apply(lambda x: monthly_me.loc[x['MONTHLY_DATE'], 4], axis = 1)
            
            # Compute the weight for each stock as its lagged ME / sum of lagged MEs in that portfolio on that month_date
            bin_4['weight'] = bin_4['SHIFTED_MONTHLY_ME'] / bin_4['MONTHLY_ME_TOTAL']


            # Bin 5

            # Get where the index is equal to the monthly date and return the value of the summed ME for that monthly date for that bin ; make a new column of this series
            # The summed ME will be used as the denominator in the weights calculations later 
            bin_high['MONTHLY_ME_TOTAL'] = bin_high.apply(lambda x: monthly_me.loc[x['MONTHLY_DATE'], 5], axis = 1)

            # Compute the weight for each stock as its lagged ME / sum of lagged MEs in that portfolio on that month_date
            bin_high['weight'] = bin_high['SHIFTED_MONTHLY_ME'] / bin_high['MONTHLY_ME_TOTAL']


            # Bin NonRD

            # Get where the index is equal to the monthly date and return the value of the summed ME for that monthly date for that bin ; make a new column of this series
            # The summed ME will be used as the denominator in the weights calculations later 
            bin_nonRD['MONTHLY_ME_TOTAL'] = bin_nonRD.apply(lambda x: monthly_me.loc[x['MONTHLY_DATE'], 'NonRD'], axis = 1)
            
            # Compute the weight for each stock as its lagged ME / sum of lagged MEs in that portfolio on that month_date
            bin_nonRD['weight'] = bin_nonRD['SHIFTED_MONTHLY_ME'] / bin_nonRD['MONTHLY_ME_TOTAL']


            # Put dataframes back together
            joined_df = pd.concat([bin_low, bin_2, bin_3, bin_4, bin_high, bin_nonRD])

            # Make new column with weight * return
            joined_df['WEIGHTED_RET'] = joined_df['RET'] * joined_df['weight']

            # Calculate portfolio returns by grouping by monthly_date and bin and taking the sum of the weighted returns
            # Creates df that is indexed by monthly_date and has columns for each bin with the total weighted return for that bin for that month
            portfolio_returns = portfolio_returns.append(joined_df.groupby(['MONTHLY_DATE', 'bin'])['WEIGHTED_RET'].sum().unstack())
        
        else: 
            print('Type must be equal or value')

 
    return portfolio_returns

### Value-Weighted Results (Remove 1000 Largest Firms)

In [34]:
# Perform for time periods
results_value_1981_1999_drop = make_portfolios_1000(merge, '1981-07-01', '1999-12-31', 'value')
results_value_2000_2012_drop = make_portfolios_1000(merge, '2000-01-01', '2012-12-31', 'value') 
results_value_2013_2021_drop = make_portfolios_1000(merge, '2013-01-01', '2021-12-31', 'value') 
results_value_1981_2012_drop = make_portfolios_1000(merge, '1981-07-01', '2012-12-31', 'value')
results_value_1981_2021_drop = make_portfolios_1000(merge, '1981-07-01', '2021-12-31', 'value')

In [35]:
# 1981 - 1999

# Collapses the portolio_returns df into a series of mean returns for each portfolio for time period
results_value_1981_1999_summary_drop = pd.DataFrame(results_value_1981_1999_drop.mean(axis = 0)).T
results_value_1981_1999_summary_drop = results_value_1981_1999_summary_drop * 100   # Converts to percentage returns 
results_value_1981_1999_summary_drop = results_value_1981_1999_summary_drop.rename(columns = {1: 'Low', 5: 'High'})
results_value_1981_1999_summary_drop['Time'] = '1981.07 - 1999.12'
# Set time to index
results_value_1981_1999_summary_drop = results_value_1981_1999_summary_drop.set_index('Time')


# 2000 - 2012

# Collapses the portolio_returns df into a series of mean returns for each portfolio for time period
results_value_2000_2012_summary_drop = pd.DataFrame(results_value_2000_2012_drop.mean(axis = 0)).T
results_value_2000_2012_summary_drop = results_value_2000_2012_summary_drop * 100
results_value_2000_2012_summary_drop = results_value_2000_2012_summary_drop.rename(columns = {1: 'Low', 5: 'High'})
results_value_2000_2012_summary_drop['Time'] = "2000.01 - 2012.12"
# Set time to index
results_value_2000_2012_summary_drop = results_value_2000_2012_summary_drop.set_index('Time')


# 1981 - 2012

# Collapses the portolio_returns df into a series of mean returns for each portfolio for time period
results_value_1981_2012_summary_drop = pd.DataFrame(results_value_1981_2012_drop.mean(axis = 0)).T
results_value_1981_2012_summary_drop = results_value_1981_2012_summary_drop * 100
results_value_1981_2012_summary_drop = results_value_1981_2012_summary_drop.rename(columns = {1: 'Low', 5: 'High'})
results_value_1981_2012_summary_drop['Time'] = "1981.07 - 2012.12"
# Set time to index
results_value_1981_2012_summary_drop = results_value_1981_2012_summary_drop.set_index('Time')

In [36]:
# Concatenate all dataframes
results_value_tog_drop = pd.concat([results_value_1981_2012_summary_drop, results_value_1981_1999_summary_drop, results_value_2000_2012_summary_drop])
# results_value_tog_drop.to_csv('results_value_tog_drop.csv')
results_value_tog_drop

bin,Low,2,3,4,High,NonRD
Time,,,,,,
1981.07 - 2012.12,0.317333,0.536762,0.740594,0.872438,1.290774,0.580132
1981.07 - 1999.12,0.406568,0.604770,0.853944,1.015888,1.277229,0.496166
2000.01 - 2012.12,0.148039,0.443561,0.580614,0.669582,1.307938,0.701900


### Long-Short Portfolio - CAPM Alpha, FF3 Alpha, and SR (Remove 1000 Largest Firms)

#### (i) CAPM Alpha

In [37]:
# Rename cols, get zero cost portfolio (High - Low)
results_value_1981_2012_drop = results_value_1981_2012_drop.rename(columns = {1: 'Low', 5: 'High'})
results_value_1981_2012_drop['H-L'] = results_value_1981_2012_drop['High'] - results_value_1981_2012_drop['Low']
# Create Year-Month column to merge fama data with 
results_value_1981_2012_drop['YEAR_MONTH'] = results_value_1981_2012_drop.index.year.astype(str) + '-' + results_value_1981_2012_drop.index.month.astype(str).str.zfill(2)

# Merge fama and portfolio returns
merged_value_fama_drop = results_value_1981_2012_drop.merge(fama_data, left_on = 'YEAR_MONTH', right_on = 'DATE')
merged_value_fama_drop = merged_value_fama_drop.set_index('YEAR_MONTH')

In [38]:
# Regressing the H-L on the Mkt-RF
X_v_drop = merged_value_fama_drop['Mkt-RF']
y_v_drop = merged_value_fama_drop['H-L']
X_v_drop = sm.add_constant(X_v_drop)
model_v_drop = sm.OLS(y_v_drop, X_v_drop).fit()
print(model_v_drop.summary())

# T stat and coeff on the constant
t_stat_v_drop = model_v_drop.tvalues[0]
coef_v_drop = model_v_drop.params[0]

                            OLS Regression Results                            
Dep. Variable:                    H-L   R-squared:                       0.045
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     17.53
Date:                Tue, 22 Nov 2022   Prob (F-statistic):           3.52e-05
Time:                        19:20:26   Log-Likelihood:                 651.62
No. Observations:                 378   AIC:                            -1299.
Df Residuals:                     376   BIC:                            -1291.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0086      0.002      3.821      0.0

In [39]:
print('Value Weighted Portfolio Results (1981-2012) with top 1000 Firms Dropped')
print()
print('Monthly CAPM Alpha: ' + str(round(coef_v_drop * 100, 3)) + '%')
print('t-statistic:', round(t_stat_v_drop, 3))

Value Weighted Portfolio Results (1981-2012) with top 1000 Firms Dropped

Monthly CAPM Alpha: 0.857%
t-statistic: 3.821


#### (ii) FF3 Alpha

In [40]:
# FF3 Factor alpha
# Regressing H-L on Mkt-RF, SMB, and HML
X_v_2_drop = merged_value_fama_drop[['Mkt-RF', 'SMB', 'HML']]
y_v_2_drop = merged_value_fama_drop['H-L']
X_v_2_drop = sm.add_constant(X_v_2_drop)
model_v_2_drop = sm.OLS(y_v_2_drop, X_v_2_drop).fit()
print(model_v_2_drop.summary())

# T stat and coeff on the constant
t_stat_v_2_drop = model_v_2_drop.tvalues[0]
coef_v_2_drop = model_v_2_drop.params[0]

                            OLS Regression Results                            
Dep. Variable:                    H-L   R-squared:                       0.150
Model:                            OLS   Adj. R-squared:                  0.143
Method:                 Least Squares   F-statistic:                     21.99
Date:                Tue, 22 Nov 2022   Prob (F-statistic):           3.88e-13
Time:                        19:20:26   Log-Likelihood:                 673.71
No. Observations:                 378   AIC:                            -1339.
Df Residuals:                     374   BIC:                            -1324.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0080      0.002      3.717      0.0

In [41]:
print('Value Weighted Portfolio Results (1981-2012) with top 1000 Firms Dropped')
print()
print('Monthly FF3 Alpha: ' + str(round(coef_v_2_drop * 100, 3)) + '%')
print('t-statistic:', round(t_stat_v_2_drop, 3))

Value Weighted Portfolio Results (1981-2012) with top 1000 Firms Dropped

Monthly FF3 Alpha: 0.799%
t-statistic: 3.717


#### (iii) Annualized Sharpe Ratio

In [42]:
# Series of zero cost monthly portfolio returns
long_short_drop = merged_value_fama_drop['H-L']

# Get mean and stdev of those returns
mean_return_drop = long_short_drop.mean()
std_dev_drop = long_short_drop.std()

# Monthly and annualized SR
monthly_sharpe_ratio_drop = mean_return_drop / std_dev_drop
annualized_sharpe_ratio_drop = monthly_sharpe_ratio_drop * np.sqrt(12)
print('Annualized Sharpe Ratio: ', round(annualized_sharpe_ratio_drop, 3))

Annualized Sharpe Ratio:  0.763


<br>

### **5. Repeat Steps 1 - 4 through December 2021**

#### 1. Calculate Equal-Weighted Returns for R&D and Non-R&D Firms.

In [43]:
# 1981 - 2021

# Collapses the portolio_returns df into a series of mean returns for each portfolio for time period
results_equal_1981_2021_summary = pd.DataFrame(results_equal_1981_2021.mean(axis = 0)).T
results_equal_1981_2021_summary = results_equal_1981_2021_summary * 100
results_equal_1981_2021_summary = results_equal_1981_2021_summary.rename(columns = {1: 'Low', 5: 'High'})
results_equal_1981_2021_summary['Time'] = '1981.07 - 2021.12'
# Set time to index
results_equal_1981_2021_summary = results_equal_1981_2021_summary.set_index('Time')


# 2013 - 2021

# Collapses the portolio_returns df into a series of mean returns for each portfolio for time period
results_equal_2013_2021_summary = pd.DataFrame(results_equal_2013_2021.mean(axis = 0)).T
results_equal_2013_2021_summary = results_equal_2013_2021_summary * 100
results_equal_2013_2021_summary = results_equal_2013_2021_summary.rename(columns = {1: 'Low', 5: 'High'})
results_equal_2013_2021_summary['Time'] = '2013.01 - 2021.12'
# Set time to index
results_equal_2013_2021_summary = results_equal_2013_2021_summary.set_index('Time')


# Concatenate all dataframes
results_equal_tog2 = pd.concat([results_equal_1981_2021_summary, results_equal_1981_1999_summary, results_equal_2000_2012_summary, results_equal_2013_2021_summary])
# results_equal_tog2.to_csv('results_equal_tog21.csv')
results_equal_tog2

bin,Low,2,3,4,High,NonRD
Time,,,,,,
1981.07 - 2021.12,0.393994,0.715339,1.033662,1.188898,1.722929,0.761205
1981.07 - 1999.12,0.242229,0.598276,0.865817,1.122465,1.722986,0.569318
2000.01 - 2012.12,0.247645,0.581299,0.873003,1.113901,1.836796,0.900588
2013.01 - 2021.12,0.917313,1.144521,1.609046,1.427567,1.559924,0.954307


<br>

#### 2. Long-Short Portfolio - CAPM Alpha, FF3 Alpha, and SR (Equal-Weighted)

#### (i) CAPM Alpha

In [44]:
# Rename cols and get zero cost portfolio (High - Low)
results_equal_1981_2021 = results_equal_1981_2021.rename(columns = {1: 'Low', 5: 'High'})
results_equal_1981_2021['H-L'] = results_equal_1981_2021['High'] - results_equal_1981_2021['Low']
# Create Year-Month column to merge fama data with  
results_equal_1981_2021['YEAR_MONTH'] = results_equal_1981_2021.index.year.astype(str) + '-' + results_equal_1981_2021.index.month.astype(str).str.zfill(2)

# Merge fama and portfolio returns
merged_equal_fama_5 = results_equal_1981_2021.merge(fama_data, left_on = 'YEAR_MONTH', right_on = 'DATE')
merged_equal_fama_5 = merged_equal_fama_5.set_index('YEAR_MONTH')

In [45]:
# Regressing the H-L on the Mkt-RF
X_5 = merged_equal_fama_5['Mkt-RF']
y_5 = merged_equal_fama_5['H-L']
X_5 = sm.add_constant(X_5)
model_5 = sm.OLS(y_5, X_5).fit()
print(model_5.summary())

# T stat and coeff on the constant
t_stat_5 = model_5.tvalues[0]
coef_5 = model_5.params[0]

                            OLS Regression Results                            
Dep. Variable:                    H-L   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     5.789
Date:                Tue, 22 Nov 2022   Prob (F-statistic):             0.0165
Time:                        19:20:26   Log-Likelihood:                 782.85
No. Observations:                 486   AIC:                            -1562.
Df Residuals:                     484   BIC:                            -1553.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0124      0.002      5.577      0.0

In [46]:
print('Equally Weighted Portfolio Results (1981-2021)')
print()
print('Monthly CAPM Alpha: ' + str(round(coef_5 * 100, 3)) + '%')
print('t-statistic:', round(t_stat_5, 3))

Equally Weighted Portfolio Results (1981-2021)

Monthly CAPM Alpha: 1.242%
t-statistic: 5.577


#### (ii) FF3 Alpha

In [47]:
# FF3 Factor alpha
# Regressing H-L on Mkt-RF, SMB, and HML
X2_5 = merged_equal_fama_5[['Mkt-RF', 'SMB', 'HML']]
y2_5 = merged_equal_fama_5['H-L']
X2_5 = sm.add_constant(X2_5)
model2_5 = sm.OLS(y2_5, X2_5).fit()
print(model2_5.summary())

# T stat and coeff on the constant
t_stat2_5 = model2_5.tvalues[0]
coef2_5 = model2_5.params[0]

                            OLS Regression Results                            
Dep. Variable:                    H-L   R-squared:                       0.229
Model:                            OLS   Adj. R-squared:                  0.225
Method:                 Least Squares   F-statistic:                     47.83
Date:                Tue, 22 Nov 2022   Prob (F-statistic):           4.50e-27
Time:                        19:20:27   Log-Likelihood:                 843.28
No. Observations:                 486   AIC:                            -1679.
Df Residuals:                     482   BIC:                            -1662.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0127      0.002      6.426      0.0

In [48]:
print('Equally Weighted Portfolio Results (1981-2021)')
print()
print('Monthly FF3 Alpha: ' + str(round(coef2_5 * 100, 3)) + '%')
print('t-statistic:', round(t_stat2_5, 3))

Equally Weighted Portfolio Results (1981-2021)

Monthly FF3 Alpha: 1.273%
t-statistic: 6.426


#### (iii) Annualized Sharpe Ratio

In [49]:
# Series of zero cost monthly portfolio returns
long_short_5 = merged_equal_fama_5['H-L']

# Get mean and stdev of those returns
mean_return_5 = long_short_5.mean()
std_dev_5 = long_short_5.std()

# Monthly and annualized SR
monthly_sharpe_ratio_5 = mean_return_5 / std_dev_5
annualized_sharpe_ratio_5 = monthly_sharpe_ratio_5 * np.sqrt(12)

print('Annualized Sharpe Ratio:', round(annualized_sharpe_ratio_5, 3))

Annualized Sharpe Ratio: 0.946


<br>

#### 3. Repeat Steps 1, 2 for Value-Weighted Portfolios (Returns and Alphas)

#### Value-Weighted Returns

In [50]:
# 1981 - 2021

# Collapses the portolio_returns df into a series of mean returns for each portfolio for time period
results_value_1981_2021_summary = pd.DataFrame(results_value_1981_2021.mean(axis = 0)).T
results_value_1981_2021_summary = results_value_1981_2021_summary * 100
results_value_1981_2021_summary = results_value_1981_2021_summary.rename(columns = {1: 'Low', 5: 'High'})
results_value_1981_2021_summary['Time'] = '1981.07 - 2021.12'
# Set time to index
results_value_1981_2021_summary = results_value_1981_2021_summary.set_index('Time')


# 2013 - 2021

# Collapses the portolio_returns df into a series of mean returns for each portfolio for time period
results_value_2013_2021_summary = pd.DataFrame(results_value_2013_2021.mean(axis = 0)).T
results_value_2013_2021_summary = results_value_2013_2021_summary * 100
results_value_2013_2021_summary = results_value_2013_2021_summary.rename(columns = {1: 'Low', 5: 'High'})
results_value_2013_2021_summary['Time'] = '2013.01 - 2021.12'
# Set time to index
results_value_2013_2021_summary = results_value_2013_2021_summary.set_index('Time')


# Concatenate all dataframes
results_value_tog2_5 = pd.concat([results_value_1981_2021_summary, results_value_1981_1999_summary, results_value_2000_2012_summary, results_value_2013_2021_summary])
# results_value_tog2_5.to_csv('results_value_tog21_5.csv')
results_value_tog2_5

bin,Low,2,3,4,High,NonRD
Time,,,,,,
1981.07 - 2021.12,0.552896,0.826087,0.858163,1.058458,1.127225,0.630808
1981.07 - 1999.12,0.675160,0.994105,1.055652,1.082074,1.193141,0.767481
2000.01 - 2012.12,0.100742,0.055259,0.109926,0.619786,0.646435,0.243628
2013.01 - 2021.12,0.951615,1.593113,1.534927,1.659034,1.688513,0.909129


#### (i) CAPM Alpha

In [51]:
# Rename cols and get zero cost portfolio (High - Low)
results_value_1981_2021 = results_value_1981_2021.rename(columns = {1: 'Low', 5: 'High'})
results_value_1981_2021['H-L'] = results_value_1981_2021['High'] - results_value_1981_2021['Low']
# Create Year-Month column to merge fama data with
results_value_1981_2021['YEAR_MONTH'] = results_value_1981_2021.index.year.astype(str) + '-' + results_value_1981_2021.index.month.astype(str).str.zfill(2)

# Merge fama and portfolio returns
merged_value_fama_5_a = results_value_1981_2021.merge(fama_data, left_on = 'YEAR_MONTH', right_on = 'DATE')
merged_value_fama_5_a = merged_value_fama_5_a.set_index('YEAR_MONTH')

In [52]:
# Regressing the H-L on the Mkt-RF
X_5_a = merged_value_fama_5_a['Mkt-RF']
y_5_a = merged_value_fama_5_a['H-L']
X_5_a = sm.add_constant(X_5_a)
model_5_a = sm.OLS(y_5_a, X_5_a).fit()
print(model_5_a.summary())

# T stat and coeff on the constant
t_stat_5_a = model_5_a.tvalues[0]
coef_5_a = model_5_a.params[0]

                            OLS Regression Results                            
Dep. Variable:                    H-L   R-squared:                       0.104
Model:                            OLS   Adj. R-squared:                  0.102
Method:                 Least Squares   F-statistic:                     56.29
Date:                Tue, 22 Nov 2022   Prob (F-statistic):           3.02e-13
Time:                        19:20:27   Log-Likelihood:                 802.66
No. Observations:                 486   AIC:                            -1601.
Df Residuals:                     484   BIC:                            -1593.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0031      0.002      1.465      0.1

In [53]:
print('Value Weighted Portfolio Results (1981-2021)')
print()
print('Monthly CAPM Alpha: ' + str(round(coef_5_a * 100, 3)) + '%')
print('t-statistic:', round(t_stat_5_a, 3))

Value Weighted Portfolio Results (1981-2021)

Monthly CAPM Alpha: 0.313%
t-statistic: 1.465


#### (ii) FF3 Alpha

In [54]:
# FF3 Factor alpha
# Regressing H-L on Mkt-RF, SMB, and HML
X2_5_a = merged_value_fama_5_a[['Mkt-RF', 'SMB', 'HML']]
y2_5_a = merged_value_fama_5_a['H-L']
X2_5_a = sm.add_constant(X2_5_a)
model2_5_a = sm.OLS(y2_5_a, X2_5_a).fit()
print(model2_5_a.summary())

# T stat and coeff on the constant
t_stat2_5_a = model2_5_a.tvalues[0]
coef2_5_a = model2_5_a.params[0]

                            OLS Regression Results                            
Dep. Variable:                    H-L   R-squared:                       0.343
Model:                            OLS   Adj. R-squared:                  0.339
Method:                 Least Squares   F-statistic:                     84.06
Date:                Tue, 22 Nov 2022   Prob (F-statistic):           9.34e-44
Time:                        19:20:27   Log-Likelihood:                 878.18
No. Observations:                 486   AIC:                            -1748.
Df Residuals:                     482   BIC:                            -1732.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0021      0.002      1.158      0.2

In [55]:
print('Value Weighted Portfolio Results (1981-2021)')
print()
print('Monthly FF3 Alpha: ' + str(round(coef2_5_a * 100, 3)) + '%')
print('t-statistic:', round(t_stat2_5_a, 3))

Value Weighted Portfolio Results (1981-2021)

Monthly FF3 Alpha: 0.214%
t-statistic: 1.158


#### (iii) Annualized Sharpe Ratio

In [56]:
# Series of zero cost monthly portfolio returns
long_short_5_a = merged_value_fama_5_a['H-L']

# Get mean and stdev of those returns
mean_return_5_a = long_short_5_a.mean()
std_dev_5_a = long_short_5_a.std()

# Monthly and annualized SR
monthly_sharpe_ratio_5_a = mean_return_5_a / std_dev_5_a
annualized_sharpe_ratio_5_a = monthly_sharpe_ratio_5_a * np.sqrt(12)

print('Annualized Sharpe Ratio:', round(annualized_sharpe_ratio_5_a, 3))

Annualized Sharpe Ratio: 0.405


<br>

#### 4. Repeat Steps 1, 2 for Value-Weighted Portfolios (Returns and Alphas) - Excluding 1000 Largest Firms

#### Value Weighted Returns (Remove 1000 Largest Firms)

In [57]:
# 1981 - 2021

# Collapses the portolio_returns df into a series of mean returns for each portfolio for time period
results_value_1981_2021_summary_drop = pd.DataFrame(results_value_1981_2021_drop.mean(axis = 0)).T
results_value_1981_2021_summary_drop = results_value_1981_2021_summary_drop * 100
results_value_1981_2021_summary_drop = results_value_1981_2021_summary_drop.rename(columns = {1: 'Low', 5: 'High'})
results_value_1981_2021_summary_drop['Time'] = '1981.07 - 2021.12'
# Set time to index
results_value_1981_2021_summary_drop = results_value_1981_2021_summary_drop.set_index('Time')


# 2013 - 2021

# Collapses the portolio_returns df into a series of mean returns for each portfolio for time period
results_value_2013_2021_summary_drop = pd.DataFrame(results_value_2013_2021_drop.mean(axis = 0)).T
results_value_2013_2021_summary_drop = results_value_2013_2021_summary_drop * 100
results_value_2013_2021_summary_drop = results_value_2013_2021_summary_drop.rename(columns = {1: 'Low', 5: 'High'})
results_value_2013_2021_summary_drop['Time'] = '2013.01 - 2021.12'
# Set time to index
results_value_2013_2021_summary_drop = results_value_2013_2021_summary_drop.set_index('Time')


# Concatenate all dataframes
results_value_tog2_5_drop = pd.concat([results_value_1981_2021_summary_drop, results_value_1981_1999_summary_drop, results_value_2000_2012_summary_drop, results_value_2013_2021_summary_drop])
# results_value_tog2_5_drop.to_csv('results_value_tog21_5_drop.csv')
results_value_tog2_5_drop

bin,Low,2,3,4,High,NonRD
Time,,,,,,
1981.07 - 2021.12,0.484157,0.703383,0.890791,1.069143,1.450758,0.614514
1981.07 - 1999.12,0.406568,0.604770,0.853944,1.015888,1.277229,0.496166
2000.01 - 2012.12,0.148039,0.443561,0.580614,0.669582,1.307938,0.701900
2013.01 - 2021.12,1.076466,1.287528,1.413665,1.758713,2.011405,0.735221


#### (i) CAPM Alpha

In [58]:
# Rename cols and get zero cost portfolio (High - Low)
results_value_1981_2021_drop = results_value_1981_2021_drop.rename(columns = {1: 'Low', 5: 'High'})
results_value_1981_2021_drop['H-L'] = results_value_1981_2021_drop['High'] - results_value_1981_2021_drop['Low']
# Create Year-Month column to merge fama data with
results_value_1981_2021_drop['YEAR_MONTH'] = results_value_1981_2021_drop.index.year.astype(str) + '-' + results_value_1981_2021_drop.index.month.astype(str).str.zfill(2)

# merge fama and portfolio returns
merged_value_fama_drop_5 = results_value_1981_2021_drop.merge(fama_data, left_on = 'YEAR_MONTH', right_on = 'DATE')
merged_value_fama_drop_5 = merged_value_fama_drop_5.set_index('YEAR_MONTH')

In [59]:
# Regressing the H-L on the Mkt-RF
X_v_drop_5 = merged_value_fama_drop_5['Mkt-RF']
y_v_drop_5 = merged_value_fama_drop_5['H-L']
X_v_drop_5 = sm.add_constant(X_v_drop_5)
model_v_drop_5 = sm.OLS(y_v_drop_5, X_v_drop_5).fit()
print(model_v_drop_5.summary())

# T stat and coeff on the constant
t_stat_v_drop_5 = model_v_drop_5.tvalues[0]
coef_v_drop_5 = model_v_drop_5.params[0]

                            OLS Regression Results                            
Dep. Variable:                    H-L   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     20.96
Date:                Tue, 22 Nov 2022   Prob (F-statistic):           5.98e-06
Time:                        19:20:27   Log-Likelihood:                 827.54
No. Observations:                 486   AIC:                            -1651.
Df Residuals:                     484   BIC:                            -1643.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0082      0.002      4.014      0.0

In [60]:
print('Value Weighted Portfolio Results (1981-2021) with top 1000 Firms Dropped')
print()
print('Monthly CAPM Alpha: ' + str(round(coef_v_drop_5 * 100, 3)) + '%')
print('t-statistic:', round(t_stat_v_drop_5, 3))

Value Weighted Portfolio Results (1981-2021) with top 1000 Firms Dropped

Monthly CAPM Alpha: 0.815%
t-statistic: 4.014


#### (ii) FF3 Alpha

In [61]:
# FF3 Factor alpha
# Regressing H-L on Mkt-RF, SMB, and HML
X_v_2_drop_5 = merged_value_fama_drop_5[['Mkt-RF', 'SMB', 'HML']]
y_v_2_drop_5 = merged_value_fama_drop_5['H-L']
X_v_2_drop_5 = sm.add_constant(X_v_2_drop_5)
model_v_2_drop_5 = sm.OLS(y_v_2_drop_5, X_v_2_drop_5).fit()
print(model_v_2_drop_5.summary())

# T stat and coeff on the constant
t_stat_v_2_drop_5 = model_v_2_drop_5.tvalues[0]
coef_v_2_drop_5 = model_v_2_drop_5.params[0]

                            OLS Regression Results                            
Dep. Variable:                    H-L   R-squared:                       0.140
Model:                            OLS   Adj. R-squared:                  0.135
Method:                 Least Squares   F-statistic:                     26.25
Date:                Tue, 22 Nov 2022   Prob (F-statistic):           9.65e-16
Time:                        19:20:27   Log-Likelihood:                 854.02
No. Observations:                 486   AIC:                            -1700.
Df Residuals:                     482   BIC:                            -1683.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0082      0.002      4.208      0.0

In [62]:
print('Value Weighted Portfolio Results (1981-2021) with top 1000 Firms Dropped')
print()
print('Monthly FF3 Alpha: ' + str(round(coef_v_2_drop_5 * 100, 3)) + '%')
print('t-statistic:', round(t_stat_v_2_drop_5, 3))

Value Weighted Portfolio Results (1981-2021) with top 1000 Firms Dropped

Monthly FF3 Alpha: 0.815%
t-statistic: 4.208


#### (iii) Annualized Sharpe Ratio

In [63]:
# Series of monthly returns for the long-short portfolio
long_short_drop_5 = merged_value_fama_drop_5['H-L']

# Mean return and stdev of returns
mean_return_drop_5 = long_short_drop_5.mean()
std_dev_drop_5 = long_short_drop_5.std()

# Monthly SR and annualized SR
monthly_sharpe_ratio_drop_5 = mean_return_drop_5 / std_dev_drop_5
annualized_sharpe_ratio_drop_5 = monthly_sharpe_ratio_drop_5 * np.sqrt(12)
print('Annualized Sharpe Ratio: ', round(annualized_sharpe_ratio_drop_5, 3))

Annualized Sharpe Ratio:  0.743


<br>